# 구조적 API 기본 연산
- DataFrame의 구성
  - Row 타입의 레코드
  - 레코드에 수행할 연산 표현식을 나타내는 여러 컬럼
- 스키마: 각 컬럼명과 데이터 타입을 정의
- 파티셔닝: DataFrame이나 Dataset이 클러스터에서 물리적으로 배치되는 형태를 정의
- 파티셔닝 스키마: 파티션을 배치하는 방법을 정의

In [0]:
path="/FileStore/tables/2015_summary.json"

In [0]:
#DataFrame 생성
df= spark.read.format('json').load(path)

In [0]:
#스키마 출력
df.printSchema()

root
-- DEST_COUNTRY_NAME: string (nullable = true)
-- ORIGIN_COUNTRY_NAME: string (nullable = true)
-- count: long (nullable = true)

## 스키마
- <strong>데이터소스에서 스키마를 얻거나 직접 정의 가능</strong>
- 데이터를 읽기 전에 스키마를 정의해야 하는지 여부는 상황에 따라 달라짐
  - ETL 작업에 스파크를 사용한다면 직접 스키마를 정의해야함
  - ETL 작업 중 데이터 타입을 알기 힘든 CSV나 JSON 등의 데이터소스를 사용하는 경우, 스키마 추론 과정에서 읽어 들인 샘플로 스키마를 결정해버릴 수 있음

In [0]:
spark.read.format('json').load(path).schema

Out[5]: StructType(List(StructField(DEST_COUNTRY_NAME,StringType,true),StructField(ORIGIN_COUNTRY_NAME,StringType,true),StructField(count,LongType,true)))

------
- 스키마는 여러 개의 StructField 타입 필드로 구성된 StructType 객체임
- StructField는 이름, 데이터 타입, 값이 null일 수 있는지 지정하는 boolean 값을 가짐

In [0]:
from pyspark.sql import types as T
#직접 스키마 정의
myManualSchema = T.StructType([
  T.StructField('DEST_COUNTRY_NAME', T.StringType(), True),
  T.StructField('ORIGIN_COUNTRY_NAME', T.StringType(), True),
  T.StructField('count', T.LongType(), False, metadata={'hello':'world'})
])

In [0]:
#스키마 적용
df2 = spark.read.format('json').schema(myManualSchema).load(path)

## 컬럼과 표현식
- 스파크의 컬럼은 <strong>표현식을 사용해 레코드 단위로 계산한 값을 단순하게 나타내는 논리적인 구조</strong>
  - 표현식으로 컬럼을 선택, 조작, 제거 가능
- 따라서 컬럼의 실젯값을 얻으려면 로우가 필요하고, 
- 로우를 얻으려면 DataFrame이 필요하다.
- DataFrame을 통하지 않으면 <strong>외부에서 컬럼에 접근할 수 없다.</strong>
- 컬럼 내용을 수정하려면 반드시 DataFrame의 스파크 트랜스포메이션을 사용해야함

### 컬럼
- col함수나 column함수를 사용하여 컬럼을 생성하고 참조

In [0]:
from pyspark.sql import functions as F
#col함수로 컬럼 생성
F.col('someColumnName')

Out[8]: Column<'someColumnName'>

------
- 컬럼이 DataFrame에 있을지 없을지는 알 수 없다.
- 컬럼은 컬럼명을 카탈로그에 저장된 정보와 비교하기 전까지 <strong>미확인</strong> 상태
- 4장에서 본 것 처럼, <strong>분석기가 동작하는 단계에서 컬럼과 테이블을 분석함</strong>

#### 명시적 컬럼 참조
- DataFrame의 컬럼은 <strong>col 메서드</strong>로 참조함
- col메서드를 사용해 <strong>명시적으로 컬럼을 정의</strong>하면 스파크는 분석기 실행 단계에서 <strong>컬럼 확인 절차를 생략함</strong>

### 표현식
- 앞서 컬럼은 표현식이라고 했음 -> 표현식이란?
 - DataFrame 레코드의 <strong>여러 값에 대한 트랜스포메이션 집합</strong>
 - 여러 컬럼명을 입력으로 받아 식별하고, '단일 값'을 만들기 위해 다양한 표현식을 각 레코드에 적용하는 함수
   - 여기서 단일 값은 Map이나 Array 같은 복합 데이터 타입일 수 있음
 - <strong>expr함수</strong>로 간단히 표현식 사용 가능
   - expr함수의 인수로 표현식을 사용하면 표현식을 분석해서 트랜스포메이션과 컬럼 참조를 알아낼 수 있음

In [0]:
F.expr("(((someCol + 5)*200) - 6) < otherCol")

Out[9]: Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

----
- 위 표현식은 <strong>실행 시점에 아래와 같은 논리적 트리로 컴파일됨</strong>

<img src="https://user-images.githubusercontent.com/44635266/77066819-2ed1ed80-6a27-11ea-9246-c6ecd40e4865.png" width=50% height=50%/>


- 어차피 실행 시점에 논리 트리로 컴파일되므로 DataFrame코드나 SQL코드나 위와 같은 표현식을 작성할 수 있고, <strong>같은 성능</strong>을 발휘함

#### DataFrame 컬럼에 접근하기
- printSchema메서드로 전체 컬럼 정보를 확인할 수 있지만 프로그래밍 방식으로 컬럼에 접근할 땐 df의 columns 속성을 사용

In [0]:
df.columns

Out[12]: ['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

## 레코드와 로우
- 스파크에서 DataFrame의 각 로우는 하나의 레코드
- 스파크는 레코드를 Row 객체로 표현
- Row 객체는 내부에 바이트 배열을 가짐
- 바이트 배열 인터페이스는 오직 컬럼 표현식으로만 다룰 수 있으므로 사용자에게 절대 노출되지 않음

In [0]:
#first메서드로 로우 확인
df.first()

Out[13]: Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

### 로우 생성하기
- <strong>Row 객체는 스키마 정보를 갖고 있지 않음</strong>
- DataFrame만 유일하게 스키마를 가짐
- 따라서 Row객체를 직접 생성하려면 <strong>DataFrame의 스키마와 같은 순서</strong>로 값을 명시해야함

In [0]:
from pyspark.sql import Row

myRow = Row("hello", None, 1, False)

In [0]:
myRow[0]

Out[15]: 'hello'

In [0]:
myRow[2]

Out[16]: 1

## DataFrame의 트랜스포메이션

### DataFrame 생성하기

In [0]:
myManualSchema=T.StructType([
  T.StructField("some", T.StringType(), True),
  T.StructField("col", T.StringType(), True),
  T.StructField("names", T.LongType(), True),
])


In [0]:
myRow = Row("hello", None,1)
myDf = spark.createDataFrame([myRow], myManualSchema)
myDf.show()

+-----+----+-----+
 some| col|names|
+-----+----+-----+
hello|null| 1|
+-----+----+-----+

### select와 selectExpr
- select와 selectExpr메서드를 사용하면 데이터 테이블에 SQL을 실행하는 것 처럼 DataFrame에서도 SQL 사용 가능

In [0]:
#select dest_country_name from table limit 2
df.select("DEST_COUNTRY_NAME").show(2)

+-----------------+
DEST_COUNTRY_NAME|
+-----------------+
 United States|
 United States|
+-----------------+
only showing top 2 rows

In [0]:
#select dest_country_name, origin_country_name from table limit 2
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)

+-----------------+-------------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
 United States| Romania|
 United States| Croatia|
+-----------------+-------------------+
only showing top 2 rows

In [0]:
#expr함수로 컬럼 참조하기
#expr함수는 단순 컬럼 참조나 문자열을 이용해 컬럼을 참조할 수 있음
df.select(F.expr("dest_country_name as destination")).show(2)

+-------------+
 destination|
+-------------+
United States|
United States|
+-------------+
only showing top 2 rows

----
- 위처럼 select 메서드에 expr함수를 사용하는 패턴을 자주 사용함

-> 이를 간단하고 효율적으로 할 수 있는 메서드가 selectExpr

#### selectExpr
- 새로운 DataFrame을 생성하는 복잡한 표현식을 간단하게 만드는 도구

In [0]:
df.selectExpr("dest_country_name as destination").show(2)

+-------------+
 destination|
+-------------+
United States|
United States|
+-------------+
only showing top 2 rows

In [0]:
#예제1: 출발지와 도착지가 같은지 나타내는 withCountry컬럼 추가
df.selectExpr('*', '(dest_country_name = origin_country_name) as withinCountry').show(2)

+-----------------+-------------------+-----+-------------+
DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
 United States| Romania| 15| false|
 United States| Croatia| 1| false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows

In [0]:
#예제2: 컬럼에 대한 집계 함수 사용
df.selectExpr("avg(count)", "count(distinct(dest_country_name))").show(2)

+-----------+---------------------------------+
 avg(count)|count(DISTINCT dest_country_name)|
+-----------+---------------------------------+
1770.765625| 132|
+-----------+---------------------------------+